In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [2]:
# загрузка датасета features.csv и data_train.csv

data = dd.read_csv('git_ignore_files/features.csv', sep='\t', blocksize=150e6)
data_train = pd.read_csv('git_ignore_files/data_train.csv')

In [3]:
# загрузка датасета на сервер

import dask.dataframe as dd
from sqlalchemy import create_engine
from dask.utils import tmpfile
import tqdm
import time



for i in data.partitions:
    with tmpfile() as f:
        try:
            db = 'mysql+pymysql://:@bainel.ru/megafon'
            i.to_sql('features', db, if_exists='append')
        except:
            db = 'mysql+pymysql://:@bainel.ru/megafon'
            i.to_sql('features', db, if_exists='append')

In [ ]:
# отделение из features.csv только абонентов из data_train.csv

list_id_train = list(data_train['id'].unique())   # список id-шников
data = data[data['id'].isin(list_id_train)].compute()

In [ ]:
# data.to_csv('files_model/id_train.csv', index=False)

In [ ]:
# data = pd.read_csv('files_model/id_train.csv')
# data_train = pd.read_csv('data_train.csv')

In [ ]:
# объединение датасетов

data_end = pd.merge_asof(data_train.sort_values('buy_time'), data.sort_values('buy_time'), on='buy_time', by='id', direction='nearest')

In [ ]:
# просмотр полученного результата

data_end

In [ ]:
# это просто пустая строка))))

In [ ]:
# сохранение объединенных данных для дальнейшего тестирования

data_end.to_csv('git_ignore_files/data_merge_asof.csv', index=False)

In [ ]:
# data = pd.read_csv('files_model/data_merge_asof.csv')

In [ ]:
# просмотр соотношения классов в датасете

data_end[data_end['target'] == 1].shape[0] / data_end.shape[0] 

In [ ]:
# отбор случайных индексов строк нулевых классов в количестве равном кол-ву строк единичных

ind_0 = np.random.choice(data_end[data_end['target'] == 0].index, size=data_end[data_end['target'] == 1].shape[0], replace=False)


In [ ]:
# получение индексов единичных классов и объединение в один список

ind_1 = data_end[data_end['target'] == 1].index
print(ind_1.shape)
ind_ = list(ind_0) + list(ind_1)
len(ind_)

In [ ]:
# получение датасета для тренировки с уравненными классами 

data_train = data_end[data_end.index.isin(list(ind_))]
data_train

In [ ]:
# сохранение в файл для дальнейшей обработки

data_train.to_csv('git_ignore_files/data_train_merge_120k.csv', index=False)